In [2]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load dataset
data = []
questions = []
with open("dataset.jsonl", "r") as f:
    for line in f:
        item = json.loads(line)
        data.append(item)
        questions.append(item["question"])

# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Encode questions
question_embeddings = model.encode(questions)

# Create FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))


c:\Users\sojebsikder\sojebsikder\project\python\playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def chatbot(query):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k=1)  # Get top-1 match

    if I[0][0] != -1:  # If a match is found
        return data[I[0][0]]["answer"]
    else:
        return "I'm sorry, I don't understand that."

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = chatbot(user_input)
    print("Bot:", response)


NameError: name 'model' is not defined